# **Para *ler* Machado de Assis**

  Neste exemplo, utilizamos *shell scripts* para análise computacional da obra de Machado.

## Obras completas

As obras são de domínio público e estão disponíveis em http://machado.mec.gov.br.

Embora [Luiz Amaral](https://github.com/luxedo) tenha publicado um [dataset](https://www.kaggle.com/datasets/luxedo/machado-de-assis) no [kaggle](https://www.kaggle.com/), contendo as obras completas, decidimos criar um *script* executável para realizar o download diretamente na página do MEC.

## Download das obras de Machado de Assis

In [2]:
cat > download_machado.sh << 'EOF'
#!/bin/bash

# Pasta onde os arquivos serão baixados e convertidos
PASTA="Machado"
mkdir -p "$PASTA"
cd "$PASTA" || exit

# Arquivo único para o corpus completo
ARQUIVO_COMPLETO="machado_de_assis_completo.txt"
> "$ARQUIVO_COMPLETO"

# URL base da página com as obras de Machado de Assis
BASE_URL="https://machado.mec.gov.br/obra-completa-lista?start="

# Função para baixar e converter obras em PDF para texto
download_obras() {
    local url="$1"
    echo "Processando página: $url"

    # Baixa a página HTML
    local pagina=$(curl -s "$url")

    # Extrai os links de download dos PDFs
    local links=$(echo "$pagina" | grep -oP '/obra-completa-lista/item/download/[^"]+' | sort -u)

    # Contador para progresso
    local total=$(echo "$links" | wc -l)
    local contador=0

    [ -z "$links" ] && echo "Nenhum link encontrado nesta página." && return

    echo "Encontrados $total arquivos nesta página"

    for link in $links; do
        contador=$((contador+1))

        # Constrói o URL completo
        local full_url="https://machado.mec.gov.br$link"

        # Extrai o nome do arquivo a partir do URL
        local file_name=$(basename "$link" | cut -d'_' -f2-)
        file_name="${file_name%.*}.pdf"  # Garante extensão .pdf

        # Nome do arquivo de texto
        local txt_file_name="${file_name%.*}.txt"

        echo -e "\n[${contador}/${total}] Processando: $file_name"

        # Verifica se já existe um arquivo de texto correspondente
        if [ -f "$txt_file_name" ]; then
            echo "Arquivo de texto já existe. Pulando conversão."
        else
            # Verifica se o arquivo PDF já existe no diretório
            if [ ! -f "$file_name" ]; then
                # Faz o download do arquivo
                echo "Baixando: $full_url"
                curl -s -L -o "$file_name" "$full_url" || {
                    echo "Erro ao baixar $full_url"
                    continue
                }
            else
                echo "Arquivo PDF já existe. Pulando download."
            fi

            # Converte o arquivo PDF para texto
            if [ -f "$file_name" ]; then
                echo "Convertendo para texto..."
                pdftotext "$file_name" "$txt_file_name" || {
                    echo "Erro ao converter $file_name"
                    continue
                }

                # Remove o arquivo PDF original se a conversão foi bem-sucedida
                rm "$file_name"
                echo "Arquivo PDF removido."
            fi
        fi

        # Adiciona o conteúdo ao arquivo completo
        if [ -f "$txt_file_name" ]; then
            # Extrai o título da primeira linha
            local titulo=$(head -n 1 "$txt_file_name" | sed 's/^[ \t]*//;s/[ \t]*$//')

            # Se o título estiver vazio, usa o nome do arquivo
            if [ -z "$titulo" ]; then
                titulo="${file_name%.*}"
            fi

            echo "Adicionando: $titulo"
            echo "## $titulo" >> "$ARQUIVO_COMPLETO"
            echo "Arquivo: $txt_file_name" >> "$ARQUIVO_COMPLETO"
            echo "" >> "$ARQUIVO_COMPLETO"
            tail -n +2 "$txt_file_name" >> "$ARQUIVO_COMPLETO"
            echo -e "\n\n" >> "$ARQUIVO_COMPLETO"
        fi
    done
}

# Realiza o download das obras em todas as páginas
# Considerando 10 páginas com 12 itens cada
for start in $(seq 0 12 120); do
    url="${BASE_URL}${start}"
    download_obras "$url"
done

echo -e "\nProcessamento concluído. Arquivo completo salvo em: $PASTA/$ARQUIVO_COMPLETO"
EOF

In [3]:
 chmod +x download_machado.sh

In [17]:
./download_machado.sh

Processando página: https://machado.mec.gov.br/obra-completa-lista?start=0
Encontrados 12 arquivos nesta página

[1/12] Processando: 621e64c83ad88459ac9c521392367e7b.pdf
Arquivo de texto já existe. Pulando conversão.
Adicionando: A crítica teatral. José de Alencar : Mãe

[2/12] Processando: c5058b5e27086aa7f3fa5babfb0fe1e7.pdf
Arquivo de texto já existe. Pulando conversão.
Adicionando: Revista Dramática

[3/12] Processando: 64ebf20b050d205ce767e347fcee4770.pdf
Arquivo de texto já existe. Pulando conversão.
Adicionando: Revista dos Teatros

[4/12] Processando: abf8892d55251c1c41a36f3a2638fdc4.pdf
Arquivo de texto já existe. Pulando conversão.
Adicionando: Idéias sobre o Teatro

[5/12] Processando: ec79144c60084d0db9c43607aea29acf.pdf
Arquivo de texto já existe. Pulando conversão.
Adicionando: O passado, o presente e o futuro da literatura

[6/12] Processando: 0af31ceb4dfe012aaa6ce7b7c9e11373.pdf
Arquivo de texto já existe. Pulando conversão.
Adicionando: Os imortais

[7/12] Processando:

## Listar os títulos baixados

In [24]:
ARQUIVO_COMPLETO=Machado/machado_de_assis_completo.txt

In [25]:
grep "^## " $ARQUIVO_COMPLETO | sort | uniq | sed 's/^## //'
echo "-----------------------------------------------------------------------"
grep "^## " $ARQUIVO_COMPLETO | sort | uniq | sed 's/^## //' | wc -l

A Constituinte perante a história, pelo sr. Homem de Mello. — Sombras e Luz, do sr. B.
A crítica teatral. José de Alencar : Mãe
A Estátua de José de Alencar
Alberto de Oliveira: Meridionais
Álvares de Azevedo: Lira dos vinte anos
A Mão e a Luva
Americanas
A morte de Francisco Otaviano
A nova geração
Ao Acaso
A Paixão de Jesus
Aquarelas
A Reforma pelo jornal
A semana
As Forcas Caudinas
Badaladas
Balas de Estalo
Bons Dias!
Carlos Jansen: Contos seletos das mil e uma noites
Carta ao Sr. Bispo do Rio de Janeiro
Carta à Redação da Imprensa Acadêmica
Cartas Fluminenses
Casa Velha
Castro Alves
Cenas da vida amazônica. por José Veríssimo.
Cherchez la femme
Comentários da semana
Compêndio da Gramática Portuguesa, por Vergueiro e Pertence. — À memória de
Contos Fluminenses
Crisálidas
Crítica teatral
Crônicas
Crônicas do Dr. Semana
Desencantos
Discursos na Academia Brasileira de Letras
Dois folhetins. Suplício de uma mulher
Dom Casmurro
Eça de Queirós
Eça de Queirós: O Primo Basílio
Eduardo Prado

## Criar um arquivo vazio

Criamos um arquivo vazio para ser utilizado como argumento do executável criado para contagem de frequência de palavras, contando as *stopwords* - palavras comuns do idioma geralmente excluídas das tarefas de análise de texto ou processamento de linguagem natural.

In [19]:
cat > nostop << 'EOF'
EOF


## Criar o arquivo shell executável

In [20]:
cat > confre.sh << 'EOF'
#!/bin/bash

# Inicia o contador de tempo
echo "Iniciando processamento..."
time {
    grep -oE '\w+' "$1" |
   #perl -CS -ne 'print lc'  #  Google Colab não respeita o locale 
    awk '{print tolower($0)}' |
    sort |
    grep -wvFf "$2" |
    uniq -c |
    sort -k1,1nr -k2 |
    sed ${3:-10}q
}
EOF

## Definir as permissões

In [21]:
chmod +x confre.sh

## Contagem da frequência de palavras

In [26]:
./confre.sh $ARQUIVO_COMPLETO nostop 20

Iniciando processamento...
  75451 a
  69367 que
  66921 de
  60774 o
  57052 e
  34383 não
  28067 se
  25059 do
  24127 um
  21989 da
  20839 é
  19763 os
  16521 uma
  15381 em
  14953 com
  14788 as
  13113 para
  12391 mas
  11921 lhe
  10961 ao

real	0m7,664s
user	0m9,842s
sys	0m0,209s


# Contagem da frequência de palavras sem *stopwords*

Como mencionado anteriormente, neste procedimento são removidas as palavras mais comuns do idioma, denominadas *stopwords*, que em geral exercem função *sintagmática*, isto é, atuam como conectores ou elementos estruturais na frase.

In [76]:
./confre.sh $ARQUIVO_COMPLETO stpw 20

Iniciando processamento...
   9148 mais
   3774 ser
   3137 tempo
   3093 casa
   3003 nada
   2854 tão
   2770 olhos
   2655 dia
   2592 homem
   2537 assim
   2483 bem
   2469 dois
   2184 menos
   2181 vida
   2135 dizer
   1995 duas
   1932 onde
   1930 dias
   1858 então
   1846 ia

real	0m8,736s
user	0m10,752s
sys	0m0,245s


# The *awk* way

Ao usarmos *awk* - linguagem orientada a padrões projetada para processamento e manipulação de texto - reduzimos um passo no *script* e melhoramos o tempo de execução.


In [29]:
# Inicia o contador de tempo
echo "Iniciando processamento..."
time {
    grep -oE '\w+' $ARQUIVO_COMPLETO |
    #perl -CS -ne 'print lc'  | 
    awk '{print tolower($0)}' | #Google Colab não respeita o locale
    awk '{cnt[$0] += 1} END {for (i in cnt) print cnt[i], i}' |
    grep -wvFf stpw |
    sort -nr |
    sed ${1:-20}q
}

Iniciando processamento...
9148 mais
3774 ser
3137 tempo
3093 casa
3003 nada
2854 tão
2770 olhos
2655 dia
2592 homem
2537 assim
2483 bem
2469 dois
2184 menos
2181 vida
2135 dizer
1995 duas
1932 onde
1930 dias
1858 então
1846 ia

real	0m3,922s
user	0m6,407s
sys	0m0,184s


### Palavras com menor frequência

In [52]:
./confre.sh $ARQUIVO_COMPLETO nostop 999999 | tail -n 12 | pr -c4 -t -w80


real	0m8,084s
user	0m10,037s
sys	0m0,227s
      1 zuavo	    	  1 zumbiam	      1 zunindo	    	  1 zurro
      1 zumbaia	    	  1 zumbindo	      1 zunzum	    	  1 zuru
      1 zumbem	    	  1 zune	      1 zurrapa	    	  1 zwinglio


# Contagem da frequência de caracteres


A ordem da frequência de caracteres varia de acordo com a língua.

Ao analisarmos obras individuais, podemos identificar em que línguas foram escritas.

### Processar múltiplos arquivos

Cada arquivo representa uma língua, em sequência:

português, italiano, espanhol, inglês, francês e alemão





In [31]:
# Lista de arquivos a serem processados
Obras=("bras_cubas.txt" "commedia.txt" "quijote.txt" "hamlet.txt"
        "les-miserables.txt" "Faust.txt"
        )

# Função para processar um arquivo e retornar os 4 caracteres mais frequentes
process_file() {
    local file="$1"
    grep -oE '\w+' "$file" |
    #perl -CS -ne 'print lc' | 
    awk '{print tolower($0)}' |
    sed 's/\(.\)/\1\n/g' |
    sed '/^$/d' |
    sort |
    uniq -c |
    sort -k1,1nr -k2 |
    head -n 4
}

# Processamento dos arquivos e impressão dos resultados
for file in "${Obras[@]}"; do
    echo "$(basename "$file")"
    process_file "$file"
    echo
done


bras_cubas.txt
  35656 a
  33732 e
  27972 o
  20532 s

commedia.txt
  46804 e
  42636 a
  40267 i
  37856 o

quijote.txt
 221982 e
 193349 a
 153361 o
 125728 s

hamlet.txt
  14960 e
  11863 t
  11218 o
   9950 a

les-miserables.txt
 332097 e
 235206 t
 206945 a
 184605 o

Faust.txt
  23249 e
  14077 n
  12284 i
  10037 h



# Contagem da frequência de advérbios


In [32]:
cat > cf_adv.sh << 'EOF' 
#!/bin/bash

grep -oE '\w+' "$1" |
#perl -CS -ne 'print lc' | 
awk '{print tolower($0)}' |
grep -E "$2$"  |
sort |
uniq -c |
sort -k1,1nr -k2 |
awk '{sum += $1} END {print sum}'
EOF

In [34]:
chmod +x cf_adv.sh

**Terminados em "ve(l|is)"**

In [35]:
./cf_adv.sh $ARQUIVO_COMPLETO "ve(l|is)"

3911


**Terminados em "mente"**

In [36]:
./cf_adv.sh $ARQUIVO_COMPLETO "mente"

10330


# Listagem de palavras por tamanho

In [38]:
grep -oE '\w+' $ARQUIVO_COMPLETO |
#perl -CS -ne 'print lc' | 
awk '{print tolower($0)}' |
grep --colour -Eow '(\w{19})' |
sort |
uniq -c

      2 constitucionalmente
      1 desencadernadamente
      2 desinteressadamente
      1 despretensiosamente
      1 despropositadamente
      1 extraconstitucional
     14 extraordinariamente
      1 extraordinàriamente
      1 impressionabilidade
      1 incompatibilizassem
      1 revolucionariamente


## Contagem de frequência de *n-gramas*

In [39]:
cat > ngramas.sh << 'EOF'
#!/bin/bash

# Verifica se os argumentos foram fornecidos
if [ "$#" -lt 2 ]; then
    echo "Uso: ./ngramas.sh <n> <arquivo> [quantidade_de_resultados]"
    echo "Exemplo para bigramas: ./ngramas.sh 2 machado_de_assis_completo.txt 10"
    exit 1
fi

N=$1
ARQUIVO=$2
SAIDA=${3:-10}  # Padrão: 10 resultados

# Remove pontuação e gera n-gramas
tr -d ';,—/.:"?!' < "$ARQUIVO" |
awk -v n="$N" '{
    split($0, palavras);
    for (i = 1; i <= length(palavras) - (n - 1); i++) {
        saida = palavras[i];
        for (j = 1; j < n; j++) {
            saida = saida " " palavras[i + j];
        }
        print saida;
    }
}' |
sort |
uniq -c |
sort -bnr |
sed "${SAIDA}q"
EOF

In [40]:
chmod +x ngramas.sh

*n* = 2

In [42]:
./ngramas.sh 2 $ARQUIVO_COMPLETO

   4839 que o
   4113 que a
   3520 que não
   3403 o que
   3112 de um
   2961 e a
   2907 que se
   2610 e o
   2321 é que
   2111 de uma


*n* = 3

In [44]:
./ngramas.sh 3 $ARQUIVO_COMPLETO

    496 Rio de Janeiro
    295 o que é
    240 com os olhos
    224 de todos os
    216 mais do que
    216 é que o
    207 é que não
    207 é o que
    206 o que se
    202 que é que


# Quantas palavras únicas existem no texto?

In [45]:
grep -oE '\w+' bras_cubas.txt |
#perl -CS -ne 'print lc' | 
awk '{print tolower($0)}' |
sort |
uniq -c |
sort -k1,1nr -k2 |
wc -l

9810


In [54]:
./confre.sh bras_cubas.txt nostop 999999 | wc -l


real	0m0,334s
user	0m0,315s
sys	0m0,032s
9811


## awk way

In [50]:
grep -oE '\w+'  bras_cubas.txt |
#perl -CS -ne 'print lc' | 
awk '{print tolower($0)}' |
sort |
uniq -c |
sort -k1,1nr -k2 |
awk 'END{print NR}'

9810


In [48]:
./confre.sh bras_cubas.txt nostop 999999 | wc -l


real	0m0,327s
user	0m0,306s
sys	0m0,032s
9811


# Quantas palavras únicas foram utilizadas apenas uma vez?

In [56]:
grep -oE '\w+' bras_cubas.txt |
#perl -CS -ne 'print lc' | 
awk '{print tolower($0)}'  |
sort |
uniq -c |
sort -k1,1nr -k2 |
grep -c '^ *1.'

5983


In [55]:
./confre.sh bras_cubas.txt nostop 999999 | grep -c '^ *1.'


real	0m0,311s
user	0m0,314s
sys	0m0,026s
5983


# Qual o vocabulário central do texto?

In [58]:
grep -oE '\w+' bras_cubas.txt |
#perl -CS -ne 'print lc' | 
awk '{print tolower($0)}' |
sort |
uniq -c |
sort -k1,1nr -k2 |
awk '$1 >= 5' |
wc -l

1496


In [57]:
./confre.sh bras_cubas.txt nostop 999999 | 
awk '$1 >= 5' |
wc -l


real	0m0,259s
user	0m0,309s
sys	0m0,024s
1497


# Lei de Zipf

In [59]:
cat > zipf_law.sh << 'EOF'
#!/bin/bash

# Calcula Zipf's Law
grep -oE '\w+' "$1" |
#perl -CS -ne 'print lc' | 
awk '{print tolower($0)}' |
sed 's/[^a-z]/\n/g' |
sort |
uniq -c |
sort -k1,1nr |
awk '{print NR, $1}' |
sed "${2:-10}q"
EOF

In [60]:
chmod +x zipf_law.sh

In [61]:
./zipf_law.sh bras_cubas.txt 10 > dados.csv
head dados.csv



1 2599
2 2203
3 2120
4 1952
5 1794
6 1165
7 989
8 979
9 744
10 688


In [78]:
python <<EOF
import pandas as pd
import matplotlib.pyplot as plt

# Lê o CSV (usando " " como delimitador e sem cabeçalho)
df = pd.read_csv('dados.csv', sep=' ', header=None, names=['Rank', 'Frequencia'])

# Plotagem (escala log-log para verificar Zipf)
plt.figure(figsize=(10, 6))
plt.loglog(df['Rank'], df['Frequencia'], 'o-', label='Dados Observados')
plt.loglog(df['Rank'], [df['Frequencia'][0]/r for r in df['Rank']], 'r--', label='Lei de Zipf (Teórico)')
plt.xlabel('Rank (log)')
plt.ylabel('Frequência (log)')
plt.title('Lei de Zipf - Memórias Póstumas de Brás Cubas')
plt.legend()
plt.grid()
plt.show()
EOF

# Estatísticas básicas

**Mínima, Máxima e Média**

In [66]:
grep -oE '\w+' $ARQUIVO_COMPLETO |
awk '{print tolower($0)}' |
sort |
uniq -c |
sort -k1,1nr -k2 |
awk 'NR == 1 { max=$1; min=$1; sum=0 } { if ($1>max) max=$1; \
if ($1<min) min=$1; sum+=$1;} \
END {printf "Min: %d\nMax: %d\nMédia: %f\n", min, max, sum/NR}'

Min: 1
Max: 75451
Média: 30.724891


**Desvio padrão**

In [67]:
grep -oE '\w+'  $ARQUIVO_COMPLETO |
awk '{print tolower($0)}' |
sort |
uniq -c |
sort -k1,1nr -k2 |
sort -n |
awk '{sum+=$1; sumsq+=$1*$1} END {print sqrt(sumsq/NR - (sum/NR)^2)}'

688.478


**Média, Mediana e Moda**

In [68]:
grep -oE '\w+'  $ARQUIVO_COMPLETO |
awk '{print tolower($0)}' |
sort |
uniq -c |
sort -k1,1nr -k2 |
sort -n |
awk '{sum+=$1;a[x++]=$1;b[$1]++}b[$1]> Mode{Mode=$1} END \
{print "Média: " sum/x "\nMediana: "a[int((x-1)/2)]"\nModa: " Mode}'

Média: 30.7249
Mediana: 2
Moda: 57


**Another way**

In [69]:
grep -oE '\w+' $ARQUIVO_COMPLETO |
awk '{print tolower($0)}' |
sort |
uniq -c |
sort -k1,1nr -k2 |
sort -n |
awk '{v[NR]=$1;sum+=$1}{if($1>max)max=$1; \
if(min>$1)min=$1}NR==1{max=min=$1}END{avg=sum/NR; \
for(n=1;n<=NR;n++)sd+=(v[n]-avg)^2;sd=sqrt(sd/NR); \
printf("Max=%f\nMin=%f\nSum=%f\nAvg=%f\nSD=%f\nItem=%d\n",max,min,sum,avg,sd,NR)}'

Max=75451.000000
Min=1.000000
Sum=1934009.000000
Avg=30.724891
SD=688.477895
Item=62946


### **Usando Rscript para calcular estatística básica**

**Opção 1**

In [70]:
grep -oE '\w+' $ARQUIVO_COMPLETO  |
awk '{print tolower($0)}' |
sort |
uniq -c |
sort -k1,1nr -k2 |
awk '{print $1}' |
Rscript -e 'd <-scan("stdin", quiet=TRUE); cat(min(d), mean(d), median(d), max(d), sd(d), sep="\n")'

1
30.72489
2
75451
688.4834


**Opção 2**

In [71]:
grep -oE '\w+' $ARQUIVO_COMPLETO |
awk '{print tolower($0)}' |
sort |
uniq -c |
sort -k1,1nr -k2 |
awk '{print $1}' |
R -q -e "x <- read.csv('stdin', header = F); summary(x); sd(x[ ,1])"

> x <- read.csv('stdin', header = F); summary(x); sd(x[ ,1])
       V1          
 Min.   :    1.00  
 1st Qu.:    1.00  
 Median :    2.00  
 Mean   :   30.72  
 3rd Qu.:    7.00  
 Max.   :75451.00  
[1] 688.4834
> 
> 


# Para localizar palavra separadas por hífen

In [72]:
awk '{print tolower($0)}' $ARQUIVO_COMPLETO |
awk '{for(i=1;i<=NF;i++){if($i~/-/){print $i}}}' |
head -10

texto-fonte:
vendeu-se,
fazer-me
sei-o,
dou-me
afastar-me
dar-lhe-ei
encontro-o
além-mar,
anuncia-se


# Concordância

In [73]:
awk '{print tolower($0)}' $ARQUIVO_COMPLETO |
grep -Eo ".{25}meneses .{25}" |
sed  -e :a -e 's/^.\{1,77\}$/ & /; ta'

          ndiscrição, félix, disse meneses ao cabo de alguns minutos          
          rsa tomou outra direção. meneses ainda tentou falar da moç          
          nada, mas triste. quando meneses entrou na sala estava ela          
          ão há indiscrição, disse meneses depois de a ver entrar em          
          filha do coronel e o dr. meneses se amassem, e eles não se          
          uetes de sua imaginação. meneses facilmente entreviu um mu          
          eição, que o espírito de meneses para logo sentiu reflorir          
          me não pode dar! repetiu meneses apegando-se ainda a uma e          
           que dava para o jardim. meneses não ousava levantar os ol          
          dados retiraram-se cedo. meneses e félix foram os últimos           
           noivo era inexplicável; meneses suspeitou a verdade e raq          
          ; pediu a intervenção de meneses para a reconciliação do m          
          o moleque abriu a porta, meneses entrou af

In [74]:
awk '{print tolower($0)}' $ARQUIVO_COMPLETO |
grep -Eo ".{25}aires .{25}" |
sed  -e :a -e 's/^.\{1,77\}$/ & /; ta'

           timas notícias de buenos aires dizem que o chefe da depu           
           io e no rosto. de buenos aires chegara-lhe na véspera, à           
           rio guaicuru, que vem em aires do casal, a palavra nioxe           
            se ele passar de buenos aires ao resto do mundo: o que            
           nte, o delírio de buenos aires chegou até cá, e o erro f           
           l da província de buenos aires por outra, mas de tirar à           
           armiento chegou a buenos-aires e tomou conta do governo,           
           ponto escasso. de buenos-aires contava eu que viesse uma           
           um convite ao voltarete. aires não teve ânimo de aceitar           
           ram visitar um ao outro; aires viria jantar às quintas-f           
           de moléstia dele, ao que aires replicou que não adoecia            
           do cansa, até a solidão. aires entrou a sentir uma ponta           
           primeira vez, porém, que aires foi a são 